Import the required libraries

In [1]:
import pandas as pd
import sklearn
import os
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_validate
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm , preprocessing
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

/home/administrator/.local/lib/python3.8/site-packages/sklearn/experimental/enable_hist_gradient_boosting.py:15: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


## Function to calculate the scores

In [2]:
def mean_score(scoring):
    return {i:j.mean() for i,j in scoring.items()}

# Loading data.
- Load the data and clean it for unique values, later shuffle it with specific seeding

In [3]:
# Get the parent directory of the current folder
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

# Specify the file path for dataset.csv in the parent folder
filename = os.path.join(parent_dir, "dataset.csv")
df = pd.read_csv(filename)

## Preprocessing data.

In [4]:
df.columns 

Index(['PhishID', 'frequency_of_a_tags', 'frequency_of_alltags',
       'presence_of_iframes', 'presence_of_popups', 'right_click_disabling',
       'checking_sfh', 'request_url', 'url-of-anchor', 'links_in_meta_img',
       'Result'],
      dtype='object')

In [5]:
data_no_duplicates = df.drop_duplicates()
df = data_no_duplicates.dropna()

In [6]:
df.shape

(6072, 11)

In [7]:
df = df.drop('PhishID', axis = 1)

As we had merged two csv files to create a dataset, we need to check the unique values present in each column

In [8]:
# Investigate unique values in the each column
unique_frequency_a = df['frequency_of_a_tags'].unique()
unique_all = df['frequency_of_alltags'].unique()
unique_iframes = df['presence_of_iframes'].unique()
unique_popups = df['presence_of_popups'].unique()
unique_rightclick = df['right_click_disabling'].unique()
# unique_redirects = df['num_of_redirects'].unique()
# unique_cookies = df['checking_cookies'].unique()
unique_sfh = df['checking_sfh'].unique()
unique_request = df['request_url'].unique()
unique_url = df['url-of-anchor'].unique()
unique_meta = df['links_in_meta_img'].unique()
unique_R = df['Result'].unique()

print(unique_frequency_a)
print(unique_all)
print(unique_iframes)
print(unique_popups)
print(unique_rightclick)
# print(unique_redirects)
# print(unique_cookies)
print(unique_sfh)
print(unique_request)
print(unique_url)
print(unique_meta)
print(unique_R)

['0' '1' '-1' 'frequency_of_a_tags']
['0' '-1' '1' 'frequency_of_alltags']
['1' 'presence_of_iframes']
['1' '0' '-1' 'presence_of_popups']
['1' 'right_click_disabling' '-1']
['-1' '1' '0' 'checking_sfh']
['-1' '1' '0' 'request_url']
['-1' '0' '1' 'url-of-anchor']
['-1' '1' '0' 'links_in_meta_img']
['1' 'Result' '-1']


Remove all the values other than {-1, 0, 1} from all the values of the dataset

In [9]:
validValues = {-1, 0, 1}

for column in df.columns:
    df[column] = pd.to_numeric(df[column], errors='coerce')
    df = df[df[column].isin(validValues)]

# Convert all columns to integers (again after the filtering)
df = df.astype(int)

In [10]:
# Now you can use the DataFrame with valid integer values (1, -1, and 0)
print(df)

      frequency_of_a_tags  frequency_of_alltags  presence_of_iframes  \
0                       0                     0                    1   
1                       1                     0                    1   
2                       1                     0                    1   
3                       1                     0                    1   
4                       0                     0                    1   
...                   ...                   ...                  ...   
6875                   -1                     0                    1   
6876                   -1                     0                    1   
6877                   -1                     0                    1   
6878                   -1                     0                    1   
6879                    0                     0                    1   

      presence_of_popups  right_click_disabling  checking_sfh  request_url  \
0                      1                      1          

Again check the unique values present in each column

In [11]:
df.shape

(6071, 10)

In [12]:
print("number of 1",len(df[df["Result"]==1]))
print("number of -1",len(df[df["Result"]==-1]))

number of 1 3406
number of -1 2665


### Data pre-processing. 
- Shuffle the data and split it into train and test sets

In [13]:
df = sklearn.utils.shuffle(df)
X = df.drop("Result",axis=1).values
X = preprocessing.scale(X)
y = df['Result'].values
df.head()

,frequency_of_a_tags,frequency_of_alltags,presence_of_iframes,presence_of_popups,right_click_disabling,checking_sfh,request_url,url-of-anchor,links_in_meta_img,Result
3890,0,0,1,1,1,-1,1,1,1,1
3789,0,0,1,1,1,-1,-1,-1,1,1
4584,0,0,1,1,1,-1,-1,-1,-1,-1
5116,-1,0,1,1,1,-1,-1,-1,-1,-1
6746,0,0,1,1,1,-1,1,1,1,-1


# Evalution metrics 
 - Specifying evaluation metrics for classification models
 - Using 10 fold-cross-validation for evaluting 

- Accuracy = (Number of Correct Predictions) / (Total Number of Predictions)
- Recall = (True Positives) / (True Positives + False Negatives)
- Precision = (True Positives) / (True Positives + False Positives)
- F1 score = 2 * (Precision * Recall) / (Precision + Recall)

In [14]:
scoring = {'accuracy': 'accuracy',
           'recall': 'recall',
           'precision': 'precision',
           'f1': 'f1'}
fold_count=10

# Decision Tree 

In [15]:
dtree_clf=DecisionTreeClassifier()
cross_val_scores = cross_validate(dtree_clf, X, y, cv=fold_count, scoring=scoring)
dtree_score = mean_score(cross_val_scores)
print(f"fit time = {dtree_score['fit_time']}")
print(f"score time = {dtree_score['score_time']}")
print(f"accuracy = {dtree_score['test_accuracy']}")
print(f"recall = {dtree_score['test_recall']}")
print(f"precision = {dtree_score['test_precision']}")
print(f"f1 = {dtree_score['test_f1']}")
print("")

fit time = 0.0029660463333129883
score time = 0.006725573539733886
accuracy = 0.7423838116708574
recall = 0.8065232016560289
precision = 0.7526492952073142
f1 = 0.7784534977302023

